In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('./raw_literacy_data.csv', low_memory=False)
pd.set_option("display.max_columns", None)

df.head()


,Country or Area,Year,Area,Sex,Age,Literacy,Record Type,Reliability,Source Year,Value,Value Footnotes
0,Albania,2011,Total,Both Sexes,10 - 14,Total,Census - de jure - complete tabulation,"Final figure, complete",2013.0,233376.0,NaN
1,Albania,2011,Total,Both Sexes,10 - 14,Literate,Census - de jure - complete tabulation,"Final figure, complete",2013.0,231653.0,NaN
2,Albania,2011,Total,Both Sexes,10 - 14,Illiterate,Census - de jure - complete tabulation,"Final figure, complete",2013.0,1723.0,NaN
3,Albania,2011,Total,Both Sexes,10 +,Total,Census - de jure - complete tabulation,"Final figure, complete",2013.0,2454948.0,NaN
4,Albania,2011,Total,Both Sexes,10 +,Literate,Census - de jure - complete tabulation,"Final figure, complete",2013.0,2387409.0,NaN


In [5]:
df.drop_duplicates()

,Country or Area,Year,Area,Sex,Age,Literacy,Record Type,Reliability,Source Year,Value,Value Footnotes
0,Albania,2011,Total,Both Sexes,10 - 14,Total,Census - de jure - complete tabulation,"Final figure, complete",2013.0,233376.0,NaN
1,Albania,2011,Total,Both Sexes,10 - 14,Literate,Census - de jure - complete tabulation,"Final figure, complete",2013.0,231653.0,NaN
2,Albania,2011,Total,Both Sexes,10 - 14,Illiterate,Census - de jure - complete tabulation,"Final figure, complete",2013.0,1723.0,NaN
3,Albania,2011,Total,Both Sexes,10 +,Total,Census - de jure - complete tabulation,"Final figure, complete",2013.0,2454948.0,NaN
4,Albania,2011,Total,Both Sexes,10 +,Literate,Census - de jure - complete tabulation,"Final figure, complete",2013.0,2387409.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
93564,62,Based on a sample taken at the time of census.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93565,63,The figures were calculated by dividing and ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93566,64,Excluding population enumerated in hotels.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93567,65,Excluding homeless persons.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df['Age'].unique()

array(['10 - 14', '10 +', '15 - 19', '20 - 24', '25 - 29', '30 - 34',
       '35 - 39', '40 - 44', '45 - 49', '50 - 54', '55 - 59', '60 - 64',
       '65 - 69', '70 - 74', '75 - 79', '80 - 84', '85 +', '6 - 10',
       '6 +', '11 - 13', '14 - 19', '75 +', '85 - 89', '90 - 94',
       '95 - 99', '100 +', 'Unknown', '80 +', '65 +', '20 - 39',
       '40 - 64', '95 +', '15 +', '60 +', '70 +', '14', '14 +', '5 +',
       '35 - 44', '45 - 54', '55 - 64', '90 +', '11 - 14', '11 +',
       '5 - 9', '12 - 14', '12 +', 'Total', '25 - 39', '40 - 54',
       '10 - 19', '20 - 34', '35 - 64', '20 - 29', '30 - 39', '40 - 49',
       '50 - 59', '60 - 69', '18 - 19', '18 +', '15 - 24', '25 - 34',
       '12 - 19', '65 - 74', '75 - 84', '50 - 64', '95 - 98', '6 - 11',
       '15 - 17', '18 - 21', '22 - 24', '35 - 49', '50 +', '15 - 29',
       '30 - 44', '45 - 59', '60 - 74', '55 +', nan], dtype=object)

In [ ]:
df = df[df['Sex'] != 'Both Sexes']
df = df.drop(columns=['Source Year','Record Type','Area','Reliability','Area','Year'])





KeyError: "['Source Year', 'Record Type', 'Area', 'Reliability', 'Area', 'Year'] not found in axis"

In [8]:
df = df[df['Age'] != 'Total']

In [9]:
# Define a function to classify the age ranges
def classify_age(age):
    if pd.isna(age):  # Handle missing values
        return "Unknown"
    age = str(age).strip()  # Convert to string and strip any whitespace
    if "+" in age:  # Handle "60+" or similar
        lower_limit = int(age.split("+")[0])
        if lower_limit >= 60:
            return ">60"
        elif lower_limit >= 15:
            return "15-59"
        else:
            return "<15"
    elif "-" in age:  # Handle ranges like "10 - 14" or "15 - 59"
        try:
            lower_limit, upper_limit = map(int, age.split("-"))
            if upper_limit < 15:
                return "<15"
            elif lower_limit >= 60:
                return ">60"
            else:
                return "15-59"
        except ValueError:
            return "Unknown"  # Catch parsing errors
    elif age.isnumeric():  # Handle individual ages like "30"
        age_numeric = int(age)
        if age_numeric < 15:
            return "<15"
        elif age_numeric <= 59:
            return "15-59"
        else:
            return ">60"
    return "Unknown"  # Default for unexpected cases like 'Total'

# Apply classification to the 'Age' column
df["Age Group"] = df["Age"].apply(classify_age)

In [ ]:
# Define a function to classify the age ranges
def classify_age(age):
    # Handle NaN or missing values
    if pd.isna(age):
        return "Unknown"

    # Ensure age is treated as a string
    age = str(age)

    if "Total" in age or "+" in age or "-" in age or age.isnumeric():
        # Handle ranges and individual ages
        if "+" in age:
            lower_limit = int(age.split("+")[0].strip())
            if lower_limit >= 60:
                return ">60"
            elif lower_limit >= 15:
                return "15-59"
            else:
                return "<15"
        elif "-" in age:
            lower_limit, upper_limit = map(int, age.split("-"))
            if upper_limit < 15:
                return "<15"
            elif lower_limit >= 60:
                return ">60"
            else:
                return "15-59"
        elif age.isnumeric():
            age_numeric = int(age)
            if age_numeric < 15:
                return "<15"
            elif age_numeric <= 59:
                return "15-59"
            else:
                return ">60"
    return "Unknown"  # Default for cases like 'Total'

# Ensure the 'Age' column is converted to string
df["Age"] = df["Age"].astype(str).apply(classify_age)


In [12]:
df = df.drop_duplicates()


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59069 entries, 99 to 93568
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Country or Area  59069 non-null  object 
 1   Sex              59002 non-null  object 
 2   Age              59069 non-null  object 
 3   Literacy         59002 non-null  object 
 4   Value            59002 non-null  float64
 5   Value Footnotes  14057 non-null  object 
 6   Age Group        59069 non-null  object 
dtypes: float64(1), object(6)
memory usage: 3.6+ MB


In [14]:
df = df.drop(columns=['Age'])

In [19]:
df = df.rename(columns={'Age Group':'Age','Value':'Population'})
df.to_csv('clean_literacy_data.csv', index=False)

In [16]:
df[df['Country or Area']== 'Indonesia']

,Country or Area,Sex,Literacy,Population,Value Footnotes,Age
35875,Indonesia,Male,Total,11662417.0,NaN,<15
35876,Indonesia,Male,Literate,11365657.0,NaN,<15
35877,Indonesia,Male,Illiterate,232858.0,NaN,<15
35878,Indonesia,Male,Unknown,63902.0,NaN,<15
35879,Indonesia,Male,Total,95994450.0,NaN,<15
...,...,...,...,...,...,...
37016,Indonesia,Female,Literate,292784.0,"23,24",>60
37017,Indonesia,Female,Illiterate,667262.0,"23,24",>60
37019,Indonesia,Female,Total,888148.0,"23,24",>60
37020,Indonesia,Female,Literate,222658.0,"23,24",>60


In [17]:
aggregated_df = df.groupby('Country or Area').agg({
    'Population': 'sum',  # Sum the population
    'Sex': 'count',       # Count the number of records for each country
    'Age': 'nunique'      # Count the number of unique age groups
}).reset_index()

# Rename columns for clarity
aggregated_df.rename(columns={
    'Population': 'Total Population',
    'Sex': 'Record Count',
    'Age': 'Unique Age Groups'
}, inplace=True)

In [18]:
aggregated_df.head()

,Country or Area,Total Population,Record Count,Unique Age Groups
0,1,0.0,0,1
1,10,0.0,0,1
2,11,0.0,0,1
3,12,0.0,0,1
4,13,0.0,0,1


In [ ]:
df.head()

,Country or Area,Sex,Population,Age
124,Åland Islands,Male,151.5,<15
125,Åland Islands,Male,787.5,<15
126,Åland Islands,Male,158.5,<15
127,Åland Islands,Male,636.0,<15
128,Åland Islands,Male,157.5,<15


In [ ]:
df.to_csv('/Users/mac/Downloads/population_gender_clean.csv', index=False)